In [1]:
# Cell 1: Import Libraries and Setup Paths
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM, SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pickle
import time

# Define paths
processed_train_path = '/root/autodl-tmp/projects/USL_NSL/dataset/processed/multi/KDDTrain_processed.csv'
processed_test_path = '/root/autodl-tmp/projects/USL_NSL/dataset/processed/multi/KDDTest_processed.csv'
train_labels_path = '/root/autodl-tmp/projects/USL_NSL/dataset/processed/multi/KDDTrain_labels.csv'
test_labels_path = '/root/autodl-tmp/projects/USL_NSL/dataset/processed/multi/KDDTest_labels.csv'

# Load class names mapping
preprocessing_path = '/root/autodl-tmp/projects/USL_NSL/dataset/processed/multi/preprocessing_objects.pkl'
with open(preprocessing_path, 'rb') as f:
    preprocessing_objects = pickle.load(f)
    class_names = preprocessing_objects['class_names']

In [2]:
# Cell 2: Load and Prepare Data
print("Loading training data...")
df_train = pd.read_csv(processed_train_path)
X = df_train.drop('multiclass_label', axis=1).values
y = df_train['multiclass_label'].values

# Create binary labels (0: normal, 1: attack)
y_binary = np.where(y == 0, 0, 1)

# Split training set and validation set (80-20)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
_, _, y_binary_train, y_binary_val = train_test_split(
    X, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

# Display number of samples for each class
class_dist = pd.Series(y).value_counts().sort_index()
print("\nClass distribution in training data:")
for class_id, count in class_dist.items():
    print(f"Class {class_id} ({class_names[class_id]}): {count} samples ({count/len(y)*100:.2f}%)")

# Display data information
print("\nDataset shapes:")
print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print("Loading complete!")

Loading training data...

Class distribution in training data:
Class 0 (Normal Traffic): 67343 samples (53.46%)
Class 1 (DOS (Denial of Service)): 45927 samples (36.46%)
Class 2 (Probe (Surveillance/Scanning)): 11656 samples (9.25%)
Class 3 (R2L (Remote to Local)): 995 samples (0.79%)
Class 4 (U2R (User to Root)): 52 samples (0.04%)

Dataset shapes:
Training set: (100778, 43)
Validation set: (25195, 43)
Loading complete!


In [3]:
# Cell 3: Train OneClassSVM for Anomaly Detection (Unsupervised Learning)
print("\nTraining OneClassSVM for anomaly detection (unsupervised learning)...")

# Extract only normal traffic samples for training
normal_indices = np.where(y_train == 0)[0]
X_normal_train = X_train[normal_indices]
print(f"Using {len(X_normal_train)} normal samples for OneClassSVM training")

# Scale the data
scaler = StandardScaler()
X_normal_train_scaled = scaler.fit_transform(X_normal_train)

# Handle class imbalance using SMOTE for binary classification
print("Applying SMOTE to handle class imbalance...")
smote = SMOTE(random_state=42)
X_train_scaled = scaler.transform(X_train)
X_resampled, y_binary_resampled = smote.fit_resample(X_train_scaled, y_binary_train)

# Train OneClassSVM with optimized parameters
start_time = time.time()
anomaly_detector = OneClassSVM(
    kernel='rbf',
    nu=0.35,           # Increased nu to improve recall
    gamma='scale',     # Kernel coefficient
    cache_size=1000,   # Increased cache size
    max_iter=1000,     # Increased max iterations
    tol=1e-4           # Set a smaller tolerance for convergence
)

anomaly_detector.fit(X_normal_train_scaled)
anomaly_train_time = time.time() - start_time
print(f"OneClassSVM training completed in {anomaly_train_time:.2f} seconds")

# Function to convert OneClassSVM predictions to binary labels
def convert_predictions(predictions):
    return np.where(predictions == 1, 0, 1)

# Evaluate anomaly detector on training set
anomaly_train_pred_raw = anomaly_detector.predict(X_train_scaled)
anomaly_train_pred = convert_predictions(anomaly_train_pred_raw)

# Calculate evaluation metrics
anomaly_train_accuracy = accuracy_score(y_binary_train, anomaly_train_pred)
anomaly_train_precision, anomaly_train_recall, anomaly_train_f1, _ = precision_recall_fscore_support(
    y_binary_train, anomaly_train_pred, average='binary'
)

# Calculate unsupervised evaluation metrics
try:
    silhouette_avg = silhouette_score(X_train_scaled, anomaly_train_pred)
    davies_bouldin_idx = davies_bouldin_score(X_train_scaled, anomaly_train_pred)
    print("\nUnsupervised Evaluation Metrics:")
    print(f"Silhouette Score: {silhouette_avg:.3f}")
    print(f"Davies-Bouldin Index: {davies_bouldin_idx:.3f}")
except:
    print("Could not calculate unsupervised metrics due to single class prediction")

# Print prediction distribution
print("\nPrediction distribution (Training Set):")
unique, counts = np.unique(anomaly_train_pred, return_counts=True)
print(dict(zip(unique, counts)))

print(f"\nAnomaly detector - Training set performance:")
print(f"Accuracy: {anomaly_train_accuracy:.4f}")
print(f"Precision: {anomaly_train_precision:.4f}")
print(f"Recall: {anomaly_train_recall:.4f}")
print(f"F1-score: {anomaly_train_f1:.4f}")

# Evaluate anomaly detector on validation set
X_val_scaled = scaler.transform(X_val)
anomaly_val_pred_raw = anomaly_detector.predict(X_val_scaled)
anomaly_val_pred = convert_predictions(anomaly_val_pred_raw)

# Calculate evaluation metrics
anomaly_val_accuracy = accuracy_score(y_binary_val, anomaly_val_pred)
anomaly_val_precision, anomaly_val_recall, anomaly_val_f1, _ = precision_recall_fscore_support(
    y_binary_val, anomaly_val_pred, average='binary'
)

print(f"\nAnomaly detector - Validation set performance:")
print(f"Accuracy: {anomaly_val_accuracy:.4f}")
print(f"Precision: {anomaly_val_precision:.4f}")
print(f"Recall: {anomaly_val_recall:.4f}")
print(f"F1-score: {anomaly_val_f1:.4f}")

# Display confusion matrix for binary classification
anomaly_val_cm = confusion_matrix(y_binary_val, anomaly_val_pred)
print("\nConfusion Matrix (Validation Set) - Binary Classification:")
print(f"True Negatives: {anomaly_val_cm[0, 0]} | False Positives: {anomaly_val_cm[0, 1]}")
print(f"False Negatives: {anomaly_val_cm[1, 0]} | True Positives: {anomaly_val_cm[1, 1]}")


Training OneClassSVM for anomaly detection (unsupervised learning)...
Using 53874 normal samples for OneClassSVM training
Applying SMOTE to handle class imbalance...


/root/miniconda3/envs/sl-nsl-env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


OneClassSVM training completed in 65.60 seconds

Unsupervised Evaluation Metrics:
Silhouette Score: 0.011
Davies-Bouldin Index: 1.459

Prediction distribution (Training Set):
{0: 23144, 1: 77634}

Anomaly detector - Training set performance:
Accuracy: 0.4813
Precision: 0.4654
Recall: 0.7703
F1-score: 0.5802

Anomaly detector - Validation set performance:
Accuracy: 0.4781
Precision: 0.4633
Recall: 0.7662
F1-score: 0.5774

Confusion Matrix (Validation Set) - Binary Classification:
True Negatives: 3061 | False Positives: 10408
False Negatives: 2742 | True Positives: 8984


In [4]:
# Cell 4: Train SVC for Attack Type Classification (Supervised Learning)
print("\nTraining SVC for attack type classification (supervised learning)...")

# Calculate class weights for attack types
attack_indices_train = np.where(y_train != 0)[0]
X_attack_train = X_train[attack_indices_train]
y_attack_train = y_train[attack_indices_train]
# Shift labels to start from 0 (1->0, 2->1, 3->2, 4->3)
y_attack_train_shifted = y_attack_train - 1

# Calculate class weights for attack types
attack_class_weights = {}
n_attack_samples = len(y_attack_train_shifted)
n_attack_classes = len(np.unique(y_attack_train_shifted))
for i in range(n_attack_classes):
    n_class = np.sum(y_attack_train_shifted == i)
    attack_class_weights[i] = n_attack_samples / (n_attack_classes * n_class)

print(f"\nAttack multiclass class weights:")
for i in range(n_attack_classes):
    print(f"Attack Class {i} (Original {i+1}, {class_names[i+1]}): {attack_class_weights[i]:.4f}")

# Scale the attack data
X_attack_train_scaled = scaler.transform(X_attack_train)

# Train SVC for attack type classification
start_time = time.time()
attack_classifier = SVC(
    kernel='rbf',
    C=10,
    gamma='scale',
    class_weight=attack_class_weights,
    probability=True
)

attack_classifier.fit(X_attack_train_scaled, y_attack_train_shifted)
attack_train_time = time.time() - start_time
print(f"Attack classifier training completed in {attack_train_time:.2f} seconds")

# Evaluate attack classifier on attack samples from training set
y_attack_train_pred = attack_classifier.predict(X_attack_train_scaled)
attack_train_accuracy = accuracy_score(y_attack_train_shifted, y_attack_train_pred)
attack_train_precision, attack_train_recall, attack_train_f1, _ = precision_recall_fscore_support(
    y_attack_train_shifted, y_attack_train_pred, average='macro'
)

print(f"\nAttack classifier - Training set performance (attack samples only):")
print(f"Accuracy: {attack_train_accuracy:.4f}")
print(f"Macro-average precision: {attack_train_precision:.4f}")
print(f"Macro-average recall: {attack_train_recall:.4f}")
print(f"Macro-average F1-score: {attack_train_f1:.4f}")

# Display classification report for attack types
attack_train_report = classification_report(y_attack_train_shifted, y_attack_train_pred)
print("\nClassification Report (Training Set) - Attack Types:")
print(attack_train_report)


Training SVC for attack type classification (supervised learning)...

Attack multiclass class weights:
Attack Class 0 (Original 1, DOS (Denial of Service)): 0.3192
Attack Class 1 (Original 2, Probe (Surveillance/Scanning)): 1.2575
Attack Class 2 (Original 3, R2L (Remote to Local)): 14.7312
Attack Class 3 (Original 4, U2R (User to Root)): 279.1905
Attack classifier training completed in 107.39 seconds

Attack classifier - Training set performance (attack samples only):
Accuracy: 0.9935
Macro-average precision: 0.8358
Macro-average recall: 0.9711
Macro-average F1-score: 0.8750

Classification Report (Training Set) - Attack Types:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     36741
           1       0.98      0.99      0.99      9325
           2       0.98      0.97      0.98       796
           3       0.38      0.93      0.54        42

    accuracy                           0.99     46904
   macro avg       0.84      0.97    

In [5]:
# Cell 5: Evaluate Hybrid Model on Validation Set
print("\nEvaluating hybrid model on validation set...")

# Step 1: Use OneClassSVM to detect anomalies
anomaly_val_pred = convert_predictions(anomaly_detector.predict(X_val_scaled))

# Step 2: For samples classified as anomalies, predict the attack type
anomaly_indices_val = np.where(anomaly_val_pred == 1)[0]
X_anomaly_val = X_val_scaled[anomaly_indices_val]

if len(anomaly_indices_val) > 0:
    y_attack_val_pred = attack_classifier.predict(X_anomaly_val)
    # Shift back to original labels (0->1, 1->2, 2->3, 3->4)
    y_attack_val_pred_shifted = y_attack_val_pred + 1
    
    # Create final predictions
    y_val_pred = np.zeros_like(y_val)
    y_val_pred[anomaly_indices_val] = y_attack_val_pred_shifted
    
    # Calculate evaluation metrics
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(y_val, y_val_pred, average='macro')
    val_report = classification_report(y_val, y_val_pred)
    val_confusion = confusion_matrix(y_val, y_val_pred)
    
    print(f"\nHybrid model - Validation set performance:")
    print(f"Accuracy: {val_accuracy:.4f}")
    print(f"Macro-average precision: {val_precision:.4f}")
    print(f"Macro-average recall: {val_recall:.4f}")
    print(f"Macro-average F1-score: {val_f1:.4f}")
    print("\nClassification Report (Validation Set):")
    print(val_report)
    print("\nConfusion Matrix (Validation Set):")
    print(val_confusion)
else:
    print("No anomalies detected in validation set by OneClassSVM.")


Evaluating hybrid model on validation set...

Hybrid model - Validation set performance:
Accuracy: 0.6882
Macro-average precision: 0.4950
Macro-average recall: 0.8542
Macro-average F1-score: 0.4815

Classification Report (Validation Set):
              precision    recall  f1-score   support

           0       0.99      0.43      0.60     13469
           1       0.71      0.99      0.83      9186
           2       0.67      0.99      0.80      2331
           3       0.09      0.96      0.16       199
           4       0.01      0.90      0.02        10

    accuracy                           0.69     25195
   macro avg       0.50      0.85      0.48     25195
weighted avg       0.85      0.69      0.70     25195


Confusion Matrix (Validation Set):
[[5758 3683 1047 1946 1035]
 [  45 9069   72    0    0]
 [   0    2 2311    2   16]
 [   0    0    0  192    7]
 [   0    0    0    1    9]]


In [6]:
# Cell 6: Evaluate Hybrid Model on Test Set
print("\nEvaluating hybrid model on test set...")

# Load test data
df_test = pd.read_csv(processed_test_path)
X_test = df_test.drop('multiclass_label', axis=1).values
y_test = df_test['multiclass_label'].values
y_binary_test = np.where(y_test == 0, 0, 1)

# Display distribution of classes in test data
test_class_dist = pd.Series(y_test).value_counts().sort_index()
print("\nClass distribution in test data:")
for class_id, count in test_class_dist.items():
    print(f"Class {class_id} ({class_names[class_id]}): {count} samples ({count/len(y_test)*100:.2f}%)")

# Scale test data
X_test_scaled = scaler.transform(X_test)

# Step 1: Use OneClassSVM to detect anomalies
anomaly_test_pred_raw = anomaly_detector.predict(X_test_scaled)
anomaly_test_pred = convert_predictions(anomaly_test_pred_raw)

# Calculate binary classification metrics
anomaly_test_accuracy = accuracy_score(y_binary_test, anomaly_test_pred)
anomaly_test_precision, anomaly_test_recall, anomaly_test_f1, _ = precision_recall_fscore_support(
    y_binary_test, anomaly_test_pred, average='binary'
)

print(f"\nAnomaly detector - Test set performance:")
print(f"Accuracy: {anomaly_test_accuracy:.4f}")
print(f"Precision: {anomaly_test_precision:.4f}")
print(f"Recall: {anomaly_test_recall:.4f}")
print(f"F1-score: {anomaly_test_f1:.4f}")

# Display confusion matrix for binary classification
anomaly_test_cm = confusion_matrix(y_binary_test, anomaly_test_pred)
print("\nConfusion Matrix (Test Set) - Binary Classification:")
print(f"True Negatives: {anomaly_test_cm[0, 0]} | False Positives: {anomaly_test_cm[0, 1]}")
print(f"False Negatives: {anomaly_test_cm[1, 0]} | True Positives: {anomaly_test_cm[1, 1]}")

# Step 2: For samples classified as anomalies, predict the attack type
anomaly_indices_test = np.where(anomaly_test_pred == 1)[0]
X_anomaly_test = X_test_scaled[anomaly_indices_test]

if len(anomaly_indices_test) > 0:
    y_attack_test_pred = attack_classifier.predict(X_anomaly_test)
    # Shift back to original labels
    y_attack_test_pred_shifted = y_attack_test_pred + 1
    
    # Create final predictions
    y_test_pred = np.zeros_like(y_test)
    y_test_pred[anomaly_indices_test] = y_attack_test_pred_shifted
    
    # Calculate evaluation metrics
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(y_test, y_test_pred, average='macro')
    test_report = classification_report(y_test, y_test_pred)
    test_confusion = confusion_matrix(y_test, y_test_pred)
    
    print(f"\nHybrid model - Test set performance:")
    print(f"Accuracy: {test_accuracy:.4f}")
    print(f"Macro-average precision: {test_precision:.4f}")
    print(f"Macro-average recall: {test_recall:.4f}")
    print(f"Macro-average F1-score: {test_f1:.4f}")
    print("\nClassification Report (Test Set):")
    print(test_report)
    print("\nConfusion Matrix (Test Set):")
    print(test_confusion)
else:
    print("No anomalies detected in test set by OneClassSVM.")


Evaluating hybrid model on test set...

Class distribution in test data:
Class 0 (Normal Traffic): 9711 samples (43.08%)
Class 1 (DOS (Denial of Service)): 7458 samples (33.08%)
Class 2 (Probe (Surveillance/Scanning)): 2421 samples (10.74%)
Class 3 (R2L (Remote to Local)): 2887 samples (12.81%)
Class 4 (U2R (User to Root)): 67 samples (0.30%)

Anomaly detector - Test set performance:
Accuracy: 0.5692
Precision: 0.5692
Recall: 1.0000
F1-score: 0.7255

Confusion Matrix (Test Set) - Binary Classification:
True Negatives: 0 | False Positives: 9711
False Negatives: 0 | True Positives: 12833

Hybrid model - Test set performance:
Accuracy: 0.4275
Macro-average precision: 0.2681
Macro-average recall: 0.5070
Macro-average F1-score: 0.3345

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      9711
           1       0.46      0.84      0.59      7458
           2       0.53      0.88      0.66      2421
       

/root/miniconda3/envs/sl-nsl-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/miniconda3/envs/sl-nsl-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/miniconda3/envs/sl-nsl-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [7]:
# Cell 7: Save Trained Models
print("\nSaving models...")
import os

# Create Paths to Save Models
model_dir = '/root/autodl-tmp/projects/USL_NSL/notebooks/multi/hybrid/models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print(f"Created directory: {model_dir}")

models = {
    'scaler': scaler,
    'anomaly_detector': anomaly_detector,
    'class_names': class_names
}

model_path = '/root/autodl-tmp/projects/USL_NSL/notebooks/multi/hybrid/models/hybrid_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(models, f)
print(f"Models saved to {model_path}")


Saving models...
Models saved to /root/autodl-tmp/projects/USL_NSL/notebooks/multi/hybrid/models/hybrid_model.pkl
